In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import prepare,analyse

hparams = {
    'npratio':4,
    'scale':'demo',
    'batch_size':10,
    'his_size':50,
    'title_size':15,
    'device':'cuda:0',
    'attrs': ['title'],
    'k': ''
}

device = torch.device(hparams['device'])

## show data

In [ ]:
_, loader_train, loader_test = prepare(hparams)

In [ ]:
a = next(iter(loader_train))
b = next(iter(loader_test))
b

In [ ]:
a['candidate_title']

### Analyze MIND Datasets
- average title length
- average abstract length
- average history length
- average impression capacity
- count of history exceeding 50
- count of empty history
- count of multi-clicked impressions 

In [ ]:
hparams['scale'] = 'large'
hparams['mode'] = 'tra'
analyse(hparams)

### tailor Data to demo size

In [ ]:
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_dev/behaviors.tsv',500)
tailorData('/home/peitian_zhang/Data/MIND/MINDsmall_train/behaviors.tsv',2000)

In [ ]:
f = open('/home/peitian_zhang/Data/MIND/MINDlarge_test/news.tsv','r',encoding='utf-8')

nid2index = {}
for line in f:
    nid,_,_,_,_,_,_,_ = line.strip("\n").split('\t')

    if nid in nid2index:
        continue
    nid2index[nid] = len(nid2index) + 1

f.close()
h = open('/home/peitian_zhang/Codes/News-Recommendation/data/dictionaries/nid2idx_large_dev.json','w',encoding='utf-8')
json.dump(nid2index,h,ensure_ascii=False)
h.close()

In [ ]:
import json
f = open('/home/peitian_zhang/Data/MIND/MINDlarge_test/behaviors.tsv','r',encoding='utf-8')

uid2index = {}
for line in f:
    _,uid,_,_,_ = line.strip("\n").split('\t')

    if uid in uid2index:
        continue
    uid2index[uid] = len(uid2index) + 1

f.close()

h = open('/home/peitian_zhang/Codes/News-Recommendation/data/dictionaries/uid2idx_large_dev.json','w',encoding='utf-8')
json.dump(uid2index,h,ensure_ascii=False)
h.close()

In [ ]:
constructVocab(['/home/peitian_zhang/Data/MIND/MINDlarge_dev/news.tsv','/home/peitian_zhang/Data/MIND/MINDlarge_test/news.tsv','/home/peitian_zhang/Data/MIND/MINDlarge_train/news.tsv'], '/home/peitian_zhang/Codes/News-Recommendation/data/dictionaries/vocab_demo_title_category_subcategory.pkl', ['title','category','subcategory'])